# **Configuração de ambiente**

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pmdarima as pm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Carregamento do dataset


## Utilização do gdown

A biblioteca `gdown` esta sendo utilizada para baixar arquivos diretamente do Google Drive. Especificamente, o arquivo com o ID `1BeB7U-Y-wN5L5XnSw-QIpph4MXd93g0V` é baixado e salvo localmente como `df_final_novas_features2.csv`. Este arquivo contém o dataset que será utilizado nas etapas subsequentes de análise e modelagem.

O código para realizar o download é o seguinte:

```python
import gdown

# URL do arquivo no Google Drive
url = 'https://drive.google.com/uc?id=1BeB7U-Y-wN5L5XnSw-QIpph4MXd93g0V'

# Baixar o arquivo
gdown.download(url, 'df_final_novas_features2.csv', quiet=False)
```

Após o download, o dataset é carregado em um DataFrame do pandas para posterior tratamento e análise.


In [2]:
# baixando o gdown para baixar o arquivo de dados

!pip install gdown

In [ ]:
import gdown

# Colocar o ID do arquivo aqui (substitua 'FILE_ID' pelo ID correto)
file_id = '1BeB7U-Y-wN5L5XnSw-QIpph4MXd93g0V'
url = f'https://drive.google.com/uc?id={file_id}'

# - https://drive.google.com/file/d/1BeB7U-Y-wN5L5XnSw-QIpph4MXd93g0V/view?usp=sharing

# Baixar o arquivo
gdown.download(url, 'df_final_novas_features2.csv', quiet=False)

In [ ]:
df = pd.read_csv('df_final_novas_features2.csv')
df.info()

# Pequeno tratamento de dados

In [ ]:
# Convert 'year' and 'month' to datetime
df['date'] = pd.to_datetime(df[['year', 'month']].assign(DAY=1))

# Aggregate 'Vl Liquido Final' by 'date' and 'Origem'
agg_df = df.groupby(['date', 'Origem'])['Vl Liquido Final'].sum().reset_index()

# Display the aggregated data
agg_df.head()

# Visualização do dataset por origem.

In [ ]:
# Plot the time series of 'Vl Liquido Final' for each 'Origem'
origem_list = agg_df['Origem'].unique()

for origem in origem_list:
    origem_df = agg_df[agg_df['Origem'] == origem]
    plt.figure(figsize=(12,6))
    plt.plot(origem_df['date'], origem_df['Vl Liquido Final'], marker='o')
    plt.title(f'Vl Liquido Final Over Time for Origem {origem}')
    plt.xlabel('Date')
    plt.ylabel('Vl Liquido Final')
    plt.grid(True)
    plt.show()

## Inicializa a lista de origens

In [ ]:
# Get the list of unique 'Origem' values
origem_list = agg_df['Origem'].unique()

# Dictionary to store models for each 'Origem'
models = {}

# Função para avaliar o modelo.

In [ ]:
# Function to evaluate the model
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    # Handle division by zero in MAPE
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_indices = y_true != 0
    mape = np.mean(np.abs((y_true[non_zero_indices] - y_pred[non_zero_indices]) / y_true[non_zero_indices])) * 100
    return mae, rmse, mape

# Treinamento por origem
Divide dados em treino e teste, plota os gráficos comparando valores reais com previsto.

In [ ]:
# Loop through each 'Origem' and build a model
for origem in origem_list:
    print(f"Processing Origem: {origem}")

    # Filter data for the current 'Origem'
    origem_df = agg_df[agg_df['Origem'] == origem].copy()

    # Set 'date' as the index
    origem_df.set_index('date', inplace=True)

    # Ensure the time series is sorted
    origem_df.sort_index(inplace=True)

    # Check if there are enough data points
    if len(origem_df) < 24:
        print(f"Not enough data for Origem {origem}. Skipping...")
        continue

    # Split into training and testing sets (80% train, 20% test)
    train_size = int(len(origem_df) * 0.8)
    train_data = origem_df.iloc[:train_size]
    test_data = origem_df.iloc[train_size:]

    # Extract the endogenous variable
    endog_train = train_data['Vl Liquido Final']
    endog_test = test_data['Vl Liquido Final']

    # Fit SARIMA model using auto_arima
    try:
        stepwise_model = pm.auto_arima(endog_train, start_p=1, start_q=1,
                                       max_p=3, max_q=3, m=12,
                                       start_P=0, seasonal=True,
                                       d=None, D=1, trace=False,
                                       error_action='ignore',
                                       suppress_warnings=True,
                                       stepwise=True)

        # Fit the model
        model = SARIMAX(endog_train, order=stepwise_model.order, seasonal_order=stepwise_model.seasonal_order)
        model_fit = model.fit(disp=False)

        # Forecast for the test period
        predictions_test = model_fit.forecast(steps=len(endog_test))

        # Forecast for the next 12 months from the end of the test set
        predictions_future = model_fit.forecast(steps=12)

        # Evaluate the model on the test set
        mae, rmse, mape = evaluate_model(endog_test, predictions_test)

        print(f"Origem {origem} - MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.2f}%")

        # Plotting
        plt.figure(figsize=(12,6))

        # Plot actual data from test set
        plt.plot(endog_test.index, endog_test, label='Valores Reais (2023)', color='blue', marker="o")

        # Plot predicted data for test set
        plt.plot(endog_test.index, predictions_test, label='Valores Previstos (Teste)', linestyle='--', color='green', marker="o")

        # Plot forecast for next 12 months
        future_dates = pd.date_range(endog_test.index[-1], periods=12, freq='M')
        plt.plot(future_dates, predictions_future, label='Previsão Futura (12 meses)', linestyle='--', color='red', marker="o")

        # Vertical line to mark the end of training
        plt.axvline(x=endog_test.index[-1], color='black', linestyle=':', label='Divisão Treino/Real')

        plt.grid(True, linestyle="-", which="major", color="black", linewidth="0.5")
        plt.title(f'Origem {origem} - Receita da Rede Gazeta (2023 até agora e previsão futura)')
        plt.xlabel('Data')
        plt.ylabel('Valor Líquido Final')

        # Customize legend for clarity
        plt.legend(loc='upper left')

        # Display plot
        plt.show()

    except Exception as e:
        print(f"An error occurred for Origem {origem}: {e}")